# Training a model
see README.md for details

Loosely following this tutorial:
https://medium.com/nlplanet/a-full-guide-to-finetuning-t5-for-text2text-and-building-a-demo-with-streamlit-c72009631887

!pip install -r requirements.txt

In [1]:
import os

os.environ["WANDB_DISABLED"] = "true"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
# from glob import glob

import zipfile

import torch
import wandb
from datasets import Dataset
from evaluate import load
from accelerate import Accelerator, DataLoaderConfiguration
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
from transformers import T5Config
from transformers import T5ForConditionalGeneration
from transformers import ByT5Tokenizer  # a "dummy" tokenizer, tokenizing into bytes
from transformers import DataCollatorForSeq2Seq
from transformers import EvalPrediction

from config import data_root, model_root, checkpoint_name
from config import token_len, annot_len

2024-07-23 13:01:26.905279: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 13:01:26.905358: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 13:01:26.907057: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-23 13:01:26.916273: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-23 13:01:28.241970: W tensorflow/compiler/tf2

/usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [2]:
dataloader_config = DataLoaderConfiguration(
    use_seedable_sampler=False,
)
accelerator = Accelerator(
    dataloader_config=dataloader_config,
    project_dir=model_root,
)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
# device = accelerator.device
torch.set_default_device(device)
# torch.cuda.is_available()
# accelerator.device
device

'cpu'

In [3]:
dfs = []
# for fname in glob(f"{data_root}/*.csv"):
#     dfs += [pd.read_csv(fname, names=["inputs", "labels"])]
with zipfile.ZipFile(f"{data_root}/data-ue.zip") as zf:
    for name in zf.namelist():
        dfs += [pd.read_csv(zf.open(name), names=["input", "label"])]
df = pd.concat(dfs, axis=0)

ds = Dataset.from_pandas(df)
ds = ds.train_test_split(test_size=0.1,shuffle=True)
ds

DatasetDict({
    train: Dataset({
        features: ['input', 'label', '__index_level_0__'],
        num_rows: 36993
    })
    test: Dataset({
        features: ['input', 'label', '__index_level_0__'],
        num_rows: 4111
    })
})

In [4]:
tokenizer = ByT5Tokenizer()
# tokenizer = AutoTokenizer.from_pretrained("google/byt5-small")

# Create a tokenization function
def preprocess_function(examples):
    model_inputs = tokenizer(examples["input"], max_length=64, truncation=True)

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["label"], max_length=64, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Apply the tokenization function to the dataset
tds = ds.map(preprocess_function, batched=True)
tds = tds.remove_columns(["input", "label"])
tds

Map:   0%|          | 0/36993 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/4111 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 36993
    })
    test: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4111
    })
})

In [5]:
exact_match_metric = load("exact_match")

def compute_exact_match(pred: EvalPrediction):
    # Convert predictions to text
    predictions = pred.predictions
    references = pred.label_ids
    
    # Decode if needed
    decoded_preds = [pred.decode(pred, skip_special_tokens=True) for pred in predictions]
    decoded_labels = [label.decode(label, skip_special_tokens=True) for label in references]
    
    # Compute exact match
    result = exact_match_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"exact_match": result["exact_match"]}


args = Seq2SeqTrainingArguments(
    output_dir=f"{model_root}/byT5-ocs-ue",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    #     gradient_checkpointing=True,
    # torch_empty_cache_steps=100,
    disable_tqdm=False,
    report_to=None,  # disable wandb.ai
    load_best_model_at_end=True,
    save_total_limit = 1,
    eval_strategy="steps",
)

def init_model():
    model = T5ForConditionalGeneration(config)
    # model = model.cuda()
    model.to(device)
    return model
    
config = T5Config.from_pretrained("t5-base")
# config.task_specific_params = {}
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)

/usr/local/lib/python3.11/dist-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [6]:
trainer = Seq2SeqTrainer(
    model_init=init_model,
    args=args,
    train_dataset=tds["train"],
    eval_dataset=tds["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_exact_match,
)

In [7]:
trainer.train()

[2024-07-23 13:01:41,200] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Step,Training Loss,Validation Loss



KeyboardInterrupt



In [ ]:
trainer.save_model(output_dir=f"{model_root}/byT5-ocs-ue-final")